In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

In [ ]:
# 시스템(assistant)에게 주어진 역할
system_message = "당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다."

# 실제로 사용자 입력 -> 모델이 답해야 하는 프롬프트
prompt = """입력 정보:
- name: {name}
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}}

# 예시
{{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
"""

In [ ]:
import json
from datasets import load_dataset


def combine_cols_to_label(example):
    # 실제 컬럼명에 맞게 수정
    label_dict = {
        "gender": example["gender"],
        "masterCategory": example["masterCategory"],
        "subCategory": example["subCategory"],
        "season": example["season"],
        "usage": example["usage"],
        "baseColour": example["baseColour"],
        "articleType": example["articleType"],
    }
    example["label"] = json.dumps(label_dict, ensure_ascii=False)
    return example

def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_message
                    }
                ],
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        # name=sample["name"] -> "name" 칼럼이 실제로 있다고 가정
                        "text": prompt.format(name=sample["productDisplayName"]),
                    },
                    {
                        "type": "image",
                        # "image_file"라는 칼럼이 있다고 가정
                        "image": sample["file_path"],
                    }
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        # combine_cols_to_label에서 만든 JSON 문자열
                        "text": sample["label"],
                    }
                ],
            },
        ],
    }

dataset = load_dataset("daje/kaggle-image-datasets", split="train")
dataset_add_label = dataset.map(combine_cols_to_label)
dataset_add_label = dataset_add_label.shuffle(seed=4242)

/root/anaconda3/envs/image_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset_add_label[0]

{'file_path': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=60x80>,
 'id': 14014,
 'gender': 'Men',
 'masterCategory': 'Apparel',
 'subCategory': 'Topwear',
 'articleType': 'Tshirts',
 'baseColour': 'White',
 'season': 'Summer',
 'year': '2012',
 'usage': 'Sports',
 'productDisplayName': 'Nike Men Solid Off-White Polo Jerseys',
 'label': '{"gender": "Men", "masterCategory": "Apparel", "subCategory": "Topwear", "season": "Summer", "usage": "Sports", "baseColour": "White", "articleType": "Tshirts"}'}

In [ ]:
formatted_dataset = [format_data(row) for row in dataset_add_label]

In [ ]:
formatted_dataset[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': '당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.'}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': '입력 정보:\n- name: Nike Men Solid Off-White Polo Jerseys\n- image: [image]\n\n위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:\n1) gender\n2) masterCategory\n3) subCategory\n4) season\n5) usage\n6) baseColour\n7) articleType\n\n출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:\n{\n  "gender": "예시값",\n  "masterCategory": "예시값",\n  "subCategory": "예시값",\n  "season": "예시값",\n  "usage": "예시값",\n  "baseColour": "예시값",\n  "articleType": "예시값"\n}\n\n# 예시\n{\n  "gender": "Men",\n  "masterCategory": "Accessories",\n  "subCategory": "Eyewear",\n  "season": "Winter",\n  "usage": "Casual",\n  "baseColour": "Blue",\n  "articleType": "Sunglasses"\n}\n\n# 주의\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=60x80>}]},
  {'role': 'assistant',


In [ ]:
from sklearn.model_selection import train_test_split

# test_size=0.1로 설정하여 전체 데이터의 10%를 테스트 세트로 분리
train_dataset, test_dataset = train_test_split(formatted_dataset,
                                             test_size=0.1,
                                             random_state=42)

In [ ]:
len(train_dataset), len(test_dataset)

(39996, 4445)

In [ ]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-VL-7B-Instruct"

# 모델과 프로세서 로드
model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)
processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.04s/it]


In [ ]:
# Preparation for inference
text = processor.apply_chat_template(
    train_dataset[2]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.<|im_end|>
<|im_start|>user
입력 정보:
- name: French Connection Women Black Sequin Clutch
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}

# 예시
{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
{"gender": "Women", "masterCategory": "Accessories", "subCategory": "Bags", "season": "Summer", "usage": "Casual", "baseColour": "Black", "articleType": "Clutches"}<|im_end|>



In [ ]:
from trl import SFTConfig
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

# 텍스트와 이미지 쌍을 인코딩하기 위한 데이터 collator 함수 정의
def collate_fn(examples):
    # 각 예제에서 텍스트와 이미지를 추출하고, 텍스트는 채팅 템플릿을 적용
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

    # 텍스트를 토크나이징하고 이미지를 처리하여 일괄 처리(batch) 형태로 변환
    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)

    # labels로 사용할 input_ids 복사본 생성 후, 패딩 토큰을 -100으로 설정하여 손실 계산 시 무시하도록 함
    labels = batch["input_ids"].clone()
    #######################
    # 패딩 토큰 손실 계산 제외 #
    #######################
    labels[labels == processor.tokenizer.pad_token_id] = -100

    # 특정 이미지 토큰 인덱스는 손실 계산에서 무시 (모델에 따라 다름)
    if isinstance(processor, Qwen2VLProcessor):
        # Qwen2VL 모델의 이미지 토큰 인덱스
        image_tokens = [151652, 151653, 151655]
    else:
        # 다른 모델에서 이미지 토큰 ID를 얻어 손실 계산에서 제외
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]
    #########################################
    # 손실 계산 시 이미지 토큰 인덱스를 무시하도록 설정 #
    #########################################
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100

    # 배치에 labels 추가 (손실 계산 시 사용)
    batch["labels"] = labels

    return batch

In [ ]:
# 단일 예시 확인
example = train_dataset[0]  # 데이터셋의 첫 번째 아이템
print("단일 예시 데이터:")
print(example)

단일 예시 데이터:
{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '입력 정보:\n- name: FNF Multi Coloured Sari\n- image: [image]\n\n위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:\n1) gender\n2) masterCategory\n3) subCategory\n4) season\n5) usage\n6) baseColour\n7) articleType\n\n출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:\n{\n  "gender": "예시값",\n  "masterCategory": "예시값",\n  "subCategory": "예시값",\n  "season": "예시값",\n  "usage": "예시값",\n  "baseColour": "예시값",\n  "articleType": "예시값"\n}\n\n# 예시\n{\n  "gender": "Men",\n  "masterCategory": "Accessories",\n  "subCategory": "Eyewear",\n  "season": "Winter",\n  "usage": "Casual",\n  "baseColour": "Blue",\n  "articleType": "Sunglasses"\n}\n\n# 주의\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n'}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=60x80 at 0x7F63E5F03F10>}]}, {'role': 'assistant', 'content': [{'

In [ ]:
# collate_fn 테스트 (배치 크기 1로)
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("이미지 픽셀 형태:", batch["pixel_values"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 363])
어텐션 마스크 형태: torch.Size([1, 363])
이미지 픽셀 형태: torch.Size([24, 1176])
레이블 형태: torch.Size([1, 363])


In [ ]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0])

입력에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        138017,  79632,   3153,      8,  42039, 126558,  45104,    101,  92031,
            14, 141274,  32077,  60039,  18411,  57835, 126605,  42905, 128618,
         97929,  54070, 142713,  78952,     13, 151645,    198, 151644,    872,
           198,  43866,  28754,  60039,    510,     12,    829,     25,    434,
         38067,  17439,   4254,  20394,    328,   2780,    198,     12,   2168,
            25,    508,   1805,   2533,  80901,  60039,  18411,  81718, 144059,
         42039,     11, 136646,    220,     22,  19969,  21329,   1376,  19391,
        128605,  93668,   4718, 141966,  17380,  57835, 126605,  33883,  55673,
         50302,    510,     16,      8,   9825,    198,     17,      8,   7341,
          6746,    198,     18,      8,   1186,   6746,    198,     19,      8,
          3200,    198,     20,      8,  10431,    198,     21,      8,   2331,
         33281,    198

In [ ]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0])

레이블에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        138017,  79632,   3153,      8,  42039, 126558,  45104,    101,  92031,
            14, 141274,  32077,  60039,  18411,  57835, 126605,  42905, 128618,
         97929,  54070, 142713,  78952,     13, 151645,    198, 151644,    872,
           198,  43866,  28754,  60039,    510,     12,    829,     25,    434,
         38067,  17439,   4254,  20394,    328,   2780,    198,     12,   2168,
            25,    508,   1805,   2533,  80901,  60039,  18411,  81718, 144059,
         42039,     11, 136646,    220,     22,  19969,  21329,   1376,  19391,
        128605,  93668,   4718, 141966,  17380,  57835, 126605,  33883,  55673,
         50302,    510,     16,      8,   9825,    198,     17,      8,   7341,
          6746,    198,     18,      8,   1186,   6746,    198,     19,      8,
          3200,    198,     20,      8,  10431,    198,     21,      8,   2331,
         33281,    19

In [ ]:
# 토큰 디코딩 예시 (입력 텍스트가 어떻게 변환되었는지 확인)
decoded_text = processor.tokenizer.decode(batch["input_ids"][0])
print("\n디코딩된 텍스트:")
print(decoded_text)


디코딩된 텍스트:
<|im_start|>system
당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.<|im_end|>
<|im_start|>user
입력 정보:
- name: FNF Multi Coloured Sari
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}

# 예시
{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
{"gender": "Women", "masterCategory": "Apparel", "subCategory": "Saree", "season": "Fall", "usage": "Ethnic", "baseColour": "Multi

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
           "q_proj",    # Query 투영 레이어
           "up_proj",   # FFN 상향 투영 레이어
           "o_proj",    # Output 투영 레이어
           "k_proj",    # Key 투영 레이어
           "down_proj", # FFN 하향 투영 레이어
           "gate_proj", # FFN 게이트 투영 레이어
           "v_proj"     # Value 투영 레이어
       ],
        task_type="CAUSAL_LM",
)


# SFTConfig를 통해 학습 설정을 정의
args = SFTConfig(
    output_dir="./kaggle_image_prediction_qkvupdg_1e-5_seperate",   # 학습된 모델과 체크포인트를 저장할 디렉터리 경로 및 리포지토리 ID
    num_train_epochs=3,                                    # 전체 학습 에포크 수 (데이터셋을 몇 번 반복할지 설정)
    per_device_train_batch_size=8,                         # 각 장비(GPU)당 사용될 배치 사이즈 (메모리와 연관됨)
    gradient_accumulation_steps=8,                         # 경사 누적 스텝 수 (이 횟수만큼 기울기를 누적한 후 업데이트)
    gradient_checkpointing=True,                           # 메모리 절약을 위한 gradient checkpointing 활성화 (메모리 최적화)
    optim="adamw_torch_fused",                             # AdamW 옵티마이저 (fused 버전 사용으로 학습 속도 향상)
    logging_steps=50,                                      # 몇 스텝마다 로그를 출력할지 설정 (여기선 50 스텝마다 로그)
    save_strategy="steps",
    save_steps=50,                                        # 매 스텝마다 체크포인트 저장 설정
    learning_rate=1e-5,                                    # 학습률 (LoRA 논문에서 추천된 값 사용)
    bf16=True,                                             # bfloat16 정밀도 사용 (메모리 절약 및 속도 향상)
    tf32=True,                                             # tf32 정밀도 사용 (NVIDIA GPU에서 학습 속도 향상)
    max_grad_norm=0.3,                                     # 기울기 클리핑을 위한 최대 기울기 값 (LoRA 논문에서 추천된 값)
    warmup_ratio=0.1,                                      # 학습 초기에 학습률을 점진적으로 올리는 warmup 비율 (LoRA 논문에서 추천된 값)
    lr_scheduler_type="cosine",                            # 일정한 학습률 스케줄러 사용 (학습률이 변하지 않음)
    # push_to_hub=True,                                    # 학습된 모델을 Hugging Face Hub에 푸시할지 여부
    report_to="tensorboard",                               # TensorBoard를 통해 학습 상태를 모니터링
    gradient_checkpointing_kwargs={"use_reentrant": False},# reentrant gradient checkpointing 설정 (비재진입 방식 사용)
    dataset_text_field="",                                 # 데이터셋에서 텍스트 필드를 위한 더미 필드 (collator에서 필요)
    dataset_kwargs={"skip_prepare_dataset": True}          # collator에서 데이터셋 전처리를 건너뛰기 위한 설정
)

# 불필요한 열 삭제하지 않도록 설정 (학습 중 사용되지 않는 열이라도 유지)
args.remove_unused_columns = False

In [ ]:
# 경고 필터 설정
import warnings
warnings.filterwarnings("ignore", message="Unrecognized keys in `rope_scaling`")
warnings.filterwarnings("ignore", message="`torch.cpu.amp.autocast")

from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    dataset_text_field="",
    peft_config=peft_config,
    tokenizer=processor.tokenizer,
)

# 학습 시작, 모델은 자동으로 허브와 출력 디렉토리에 저장됨
trainer.train()

# 모델 저장
trainer.save_model(args.output_dir)

/root/anaconda3/envs/image_train/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/root/anaconda3/envs/image_train/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/root/anaconda3/envs/image_train/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/root/anaconda3/envs/image_train/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal

Step,Training Loss
50,1.483400
100,0.585500
150,0.108100
200,0.089600
250,0.083300
300,0.077600
350,0.075500
400,0.071300
450,0.069600
500,0.067900
